# Tumor volume and survival proportions

### 1. Tumor Volume Analysis
In this section, we load the tumor volume data, perform statistical analysis (One-Way ANOVA and Tukey's HSD test), and generate the corresponding plots for tumor volume over time across different experimental groups.

##### Load the CSV file into a DataFrame and reading the data

In [ ]:
import pandas as pd

# Load Tumor Volume data
df_tv = pd.read_csv("Tumor volume.csv")
df_tv

##### Data analysis

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Prepare data for ANOVA
data = {
    'Control': df_tv.iloc[6, 1:4].values,
    'JNPs+MF': df_tv.iloc[6, 4:7].values,
    'RT': df_tv.iloc[6, 7:10].values,
    'AMF': df_tv.iloc[6, 10:13].values,
    'NIR': df_tv.iloc[6, 13:16].values,
    'JNPs+MF+RT': df_tv.iloc[6, 16:19].values,
    'JNPs+MF+AMF': df_tv.iloc[6, 19:22].values,
    'JNPs+MF+NIR': df_tv.iloc[6, 22:25].values,
    'PMT': df_tv.iloc[6, 25:28].values,
    'JNPs+MF+PMT': df_tv.iloc[6, 28:31].values,
    'JNPs+MF+RT+PMT': df_tv.iloc[6, 31:34].values
}

# Transform data into a long format
rows = []
for values in zip(data['Control'], data['JNPs+MF'], data['RT'], data['AMF'], data['NIR'], data['JNPs+MF+RT'], data['JNPs+MF+AMF'], data['JNPs+MF+NIR'], data['PMT'], data['JNPs+MF+PMT'], data['JNPs+MF+RT+PMT']):
    for group, value in zip(['Control', 'JNPs+MF', 'RT', 'AMF', 'NIR', 'JNPs+MF+RT', 'JNPs+MF+AMF', 'JNPs+MF+NIR', 'PMT', 'JNPs+MF+PMT', 'JNPs+MF+RT+PMT'], values):
        rows.append({
            'Group': group,
            'Values': value
        })

df_tv2 = pd.DataFrame(rows)

# One-way ANOVA
model = ols('Values ~ C(Group)', data=df_tv2).fit()

anova_table = sm.stats.anova_lm(model, typ=1)
print("ANOVA Table:")
print(anova_table)

# Tukey's HSD Test
tukey = pairwise_tukeyhsd(endog=df_tv2['Values'], groups=df_tv2['Group'], alpha=0.05)
print("\nTukey's HSD Test:")
print(tukey)

##### Plot the results

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from comparison import pairwise_comparison

# Define lists of groups and their respective columns
groups = ['Control', 'JNPs+MF', 'RT', 'AMF', 'NIR', 'JNPs+MF+RT', 'JNPs+MF+AMF', 'JNPs+MF+NIR', 'PMT', 'JNPs+MF+PMT', 'JNPs+MF+RT+PMT']
columns = [['Control1', 'Control2', 'Control3'], ['JNPs+MF1', 'JNPs+MF2', 'JNPs+MF3'], ['RT1', 'RT2', 'RT3'], 
           ['AMF1', 'AMF2', 'AMF3'], ['NIR1', 'NIR2', 'NIR3'], ['JNPs+MF+RT1', 'JNPs+MF+RT2', 'JNPs+MF+RT3'], 
           ['JNPs+MF+AMF1', 'JNPs+MF+AMF2', 'JNPs+MF+AMF3'], ['JNPs+MF+NIR1', 'JNPs+MF+NIR2', 'JNPs+MF+NIR3'], 
           ['PMT1', 'PMT2', 'PMT3'], ['JNPs+MF+PMT1', 'JNPs+MF+PMT2', 'JNPs+MF+PMT3'], ['JNPs+MF+RT+PMT1', 'JNPs+MF+RT+PMT2', 'JNPs+MF+RT+PMT3']]

# Calculate Means and SEMs for each group
for group, cols in zip(groups, columns):
    df_tv[f'Mean_{group}'] = df_tv[cols].mean(axis=1)
    df_tv[f'SEM_{group}'] = df_tv[cols].sem(axis=1)

# Plot the data
plt.figure(figsize=(13, 10))
colors = ['#000080', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#ff9966', '#7f7f7f', '#bcbd22', '#17becf', '#ff00ff']
markers = ['o', 's', '^', 'v', 'D', '>', '<', 'H', 'h', 'd', '.']
labels = [group for group in groups]

# Plot each group with corresponding color, marker, and label
for group, color, marker, label in zip(groups, colors, markers, labels):
    plt.errorbar(df_tv['Time(day)'], df_tv[f'Mean_{group}'], yerr=df_tv[f'SEM_{group}'], color=color, linewidth=3, capsize=6)
    plt.plot(df_tv['Time(day)'], df_tv[f'Mean_{group}'], color=color, linewidth=4, linestyle='-', marker=marker, markersize=14, label=label)

# Customize plot appearance
for cap in plt.gca().get_lines():
    cap.set_markeredgewidth(2)
    cap.set_path_effects([path_effects.Stroke(linewidth=2), path_effects.Normal()])

# Customize font properties
font_properties = {'fontweight': 'bold', 'fontsize': 28, 'fontfamily': 'Arial'}
number_font_properties = {'fontweight': 'bold', 'fontsize': 28, 'fontfamily': 'Arial'}
plt.xlabel('Days', **font_properties)
plt.ylabel('Tumor volume  (mm$^{3}$)', **font_properties)
plt.xticks(df_tv['Time(day)'], ha='center', **number_font_properties)
plt.yticks([0, 1000, 2000, 3000, 4000, 5000, 6000], **number_font_properties)
plt.tick_params(axis='both', which='both', length=12, width=4, labelsize=26)

# Customize axes and ticks
ax = plt.gca()
ax.spines['left'].set_linewidth(4)
ax.spines['bottom'].set_linewidth(4)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Collect last mean and SEM values
last_values = {f'Mean_{group}': df_tv[f'Mean_{group}'].iloc[-1] for group in groups}
last_sems = {f'SEM_{group}': df_tv[f'SEM_{group}'].iloc[-1] for group in groups}

comdf = pd.DataFrame({'all_mean': list(last_values.values()), 'all_sem': list(last_sems.values())})

# Perform pairwise comparison
pairwise_comparison(5, 10, 0, range(len(comdf)), comdf['all_mean'].tolist(), dh=0.66, barh=0.015, fs=26, bold=True, line_width=3, vertical=True)
pairwise_comparison(9, 10, 0.0379, range(len(comdf)), comdf['all_mean'].tolist(), dh=0.52, barh=0.015, fs=26, bold=True, line_width=3, vertical=True)

# Set plot limits and save the picture
plt.ylim(bottom=0)
plt.xlim(left=0)
plt.savefig('Tumor_volume_graph.tif', dpi=600, format='tif', bbox_inches='tight')
plt.show()

### 2. Survival Analysis
In this section, we load the survival data, perform Kaplan-Meier survival analysis, and generate the survival curves for different experimental groups. Additionally, log-rank tests are performed to compare survival distributions.

##### Load the CSV file into a DataFrame and reading the data

In [ ]:
df_sf = pd.read_csv("Survival proportions.csv")
df_sf

##### Data analysis and plot the results

In [ ]:
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from survival import median_survival_time

time_col, event_col, treat_col = 'Time(day)', 'dead(1)/alive(0)', 'Groups'
kmf, groups = KaplanMeierFitter(), df_sf[treat_col].unique()

plt.figure(figsize=(8, 6))
ax = plt.gca()

# Plot each group's Kaplan-Meier survival curve and print the median survival time
for i, group in enumerate(groups):
    d = df_sf[df_sf[treat_col] == group]
    kmf.fit(d[time_col], event_observed=d[event_col], label=group)
    kmf.plot(ax=ax, ci_show=False, show_censors=False, lw=3, color=colors[i], legend=False)
    median = median_survival_time(d[time_col].values, d[event_col].values)
    print(f"Median Survival for {group}: {median}")

# Perform log-rank tests between each pair of groups
for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        g1, g2 = groups[i], groups[j]
        result = logrank_test(df_sf[df_sf[treat_col] == g1][time_col], df_sf[df_sf[treat_col] == g2][time_col],
                              df_sf[df_sf[treat_col] == g1][event_col], df_sf[df_sf[treat_col] == g2][event_col])
        print(f"Log-Rank (Mantel-Cox) Test between {g1} and {g2}: p-value = {result.p_value:.5f}")

# Customize the plot appearance
plt.xlabel('Time (day)', fontdict={'fontsize': 18, 'fontweight': 'bold', 'family': 'Arial'})
plt.ylabel('Survival probability', fontdict={'fontsize': 18, 'fontweight': 'bold', 'family': 'Arial'})
plt.tick_params(axis='both', which='both', length=8, width=3, labelsize=18)
for spine in ['left', 'bottom']:
    plt.gca().spines[spine].set_linewidth(3)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.ylim(0)
plt.xlim(0)
plt.xticks(fontweight='bold', family='Arial')
plt.yticks(fontweight='bold', family='Arial')

# Show the figure and save the picture
plt.savefig('Survival_proportions_graph.tif', dpi=600, format='tif', bbox_inches='tight')
plt.show()

### 3. Combining Tumor Volume and Survival Proportions Figures
In this section, we combine the tumor volume graph and survival proportions graph into a single figure. This allows us to display both the tumor volume data and the Kaplan-Meier survival analysis side by side, making it easier to compare the results visually.

In [ ]:
# Load the saved figures
tumor_volume_ax = plt.imread('Tumor_volume_graph.tif')
survival_analysis_ax = plt.imread('Survival_proportions_graph.tif')

# Create a list to store Line2D objects for the legend
legend_elements = []

# Add elements to the legend for the tumor volume graph
groups = df_sf['Groups'].unique()
for i, group in enumerate(groups):
    legend_elements.append(plt.Line2D([0], [0], color=colors[i], lw=4, label=f"{group}"))

# Combine the plots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(17, 7))

# Display the images on the subplots
axes[0].imshow(tumor_volume_ax)
axes[1].imshow(survival_analysis_ax)

for ax in axes:
    ax.axis('off')
plt.subplots_adjust(wspace=0.05)
fig.legend(handles=legend_elements, loc='lower center', ncol=4, fontsize='large', frameon=False)

# Save the combined figure
plt.savefig('Tumor_volume_and_survival_proportions.tif', dpi=600, format='tif', bbox_inches='tight', facecolor='white')
plt.show()